In [47]:
import ase
import os
import ase.calculators.vasp
import json
import libcasm.xtal as xtal
import libcasm.configuration as casmconfig

# read prim.json
with open("inputs/prim.json", "r") as f:
    prim = casmconfig.Prim.from_dict(json.load(f))


# Conversion function from ase.Atoms to libcasm.xtal.Structure
def casm_structure_to_ase_atoms(casm_structure: xtal.Structure) -> ase.Atoms:
    """Given `xtal.Structure`, convert it to `xtal.Structure`

    Parameters
    ----------
    casm_structure : xtal.Structure

    Returns
    -------
    ase.Atoms

    """
    if len(casm_structure.mol_type()):
        raise ValueError(
            "Error: only atomic structures may be converted using " "make_ase_atoms"
        )

    symbols = casm_structure.atom_type()
    positions = casm_structure.atom_coordinate_cart().transpose()
    cell = casm_structure.lattice().column_vector_matrix().transpose()

    return ase.Atoms(
        symbols=symbols,
        positions=positions,
        cell=cell,
        pbc=True,
    )


# Conversion function from ase.Atoms to libcasm.xtal.Structure
def ase_atoms_to_casm_structure(ase_atoms: ase.Atoms) -> xtal.Structure:
    """Given `ase.Atoms`, convert it to `xtal.Structure`

    Parameters
    ----------
    ase_atoms : ase.Atoms

    Returns
    -------
    xtal.Structure

    """

    lattice = xtal.Lattice(
        column_vector_matrix=ase_atoms.get_cell().transpose(),
    )
    atom_coordinate_frac = ase_atoms.get_scaled_positions().transpose()
    atom_type = ase_atoms.get_chemical_symbols()

    return xtal.Structure(
        lattice=lattice,
        atom_coordinate_frac=atom_coordinate_frac,
        atom_type=atom_type,
    )


# read config_list.json -> containing a list of enumerated configurations
with open("inputs/config_list.json", "r") as f:
    data = json.load(f)

# convert the data into a configuration set
configuration_set = casmconfig.ConfigurationSet()
for d in data:
    config = casmconfig.Configuration.from_dict(
        data=d, supercells=casmconfig.SupercellSet(prim=prim)
    )
    configuration_set.add_configuration(config)


# Write vasp input files for each of the configurations
# Will be written to <casm_proj_name>/training_data/<enumuration_id>/<config_name>/calctype.<calctype_id>/
casm_proj_name = "SiGe"  # define casm_proj_name
enum_id = "occ"  # define enumeration_id
calctype_id = "default"  # define calctype_id

os.environ["VASP_PP_PATH"] = "/home/sesha/Projects/vasp_stuff/vasp_potentials"
setups = {"Si": "", "Ge": "_d"}

for config in configuration_set:
    # get the config_name
    config_name = os.path.join(config.supercell_name, config.configuration_id)

    # final path where INCAR, KPOINTS, POTCAR, POSCAR will be written
    config_calctype_dir = os.path.join(
        casm_proj_name, "training_data", enum_id, config_name, "calctype." + calctype_id
    )

    # make config_calctype_dir
    os.makedirs(config_calctype_dir, exist_ok=True)

    # convert the current configuration to ase_atoms
    ase_atoms = casm_structure_to_ase_atoms(config.configuration.to_structure())

    # Make an ase VaspCalculator
    vasp_calculator = ase.calculators.vasp.Vasp(
        atoms=ase_atoms, directory=config_calctype_dir, setups=setups, xc="pbe"
    )
    vasp_calculator.read_incar("inputs/INCAR") # read incars
    vasp_calculator.read_kpoints("inputs/KPOINTS") # read kpoints

    vasp_calculator.write_input(atoms=ase_atoms) # Write INCAR, KPOINTS, POTCAR, POSCAR